In [1]:
import pandas as pd
import numpy as np
data = pd.read_json('data/dataset_1.json')

data["label_bin"] = np.where(data["label"] > 1, 1, 0)
data["display_text"] = [d[1]['text'][d[1]['displayTextRangeStart']: d[1]['getDisplayTextRangeEnd']] for d in data[["text","displayTextRangeStart", "getDisplayTextRangeEnd"]].iterrows()]
print("max text length", len(data.iloc[np.argmax(data['text'].to_numpy())]['text']))
max_display_text_length = len(data.iloc[np.argmax(data['display_text'].to_numpy())]['display_text'])
print("max display text length", max_display_text_length)
data
X = data.display_text
y = data.label


max text length 286
max display text length 270


In [26]:
def get_sbert_centroid_args(sentences, num_labels:int, bert_model_name='all-distilroberta-v1'):
    from sentence_transformers import SentenceTransformer, util
    import torch
    model = SentenceTransformer(bert_model_name)
    model.max_seq_length = np.argmax(sentences)
    
    embedding_list = model.encode(X, show_progress_bar=True)
    from sklearn.cluster import KMeans
    clustering_model = KMeans(n_clusters=10, random_state=1337) 
    clustering_model.fit(embedding_list)
    cluster_assignment = clustering_model.labels_
    clustered_sentences = {}
    for sentence_id, cluster_id in enumerate(cluster_assignment):
        if cluster_id not in clustered_sentences:
            clustered_sentences[cluster_id] = []

        clustered_sentences[cluster_id].append(sentence_id)

    centroids = []
    for i in range(len(clustering_model.cluster_centers_)):
        center = clustering_model.cluster_centers_[i]
        centroid_arg = np.argmin([np.linalg.norm(cluster_item-center) for cluster_item in embedding_list[clustered_sentences[i]]])
        centroids.append(centroid_arg)
    return centroids

In [28]:
get_sbert_centroid_args(sentences=X, num_labels=100)

Batches:   0%|          | 0/204 [00:00<?, ?it/s]

[780, 582, 2, 116, 455, 585, 43, 570, 307, 59]

In [ ]:
def get_